In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from bs4 import BeautifulSoup, SoupStrainer
import sys
import string
import missingno as msno 
import datetime
import progressbar
import time
import re
import os
from numbers import Number
from selenium import webdriver

from IPython.display import display
pd.options.display.max_columns = None

### Resumen del Contenido
En el siguiente notebook, a partir de los datos escrapeados en el notebook `data_scraping_v2` se realizará una limpieza inicial de las distintas tablas de datos prévia a la transformación y creación de las distintas variables que van a ser usadas para el entrenamiento del modelo.


# Importamos los Archivos para Limpieza



In [3]:
path = './Scraped data/scraped_data_v2/'

#NBA Dataframes
df_stats_RS_PG_BS_df = pd.read_csv(path + 'stats_RS_PG_BS_df.csv',index_col=0) 
df_stats_PFF_PG_BS_df = pd.read_csv(path + 'stats_PFF_PG_BS_df.csv',index_col=0) 
df_stats_RS_TOT_BS_df = pd.read_csv(path + 'stats_RS_TOT_BS_df.csv',index_col=0) 
df_stats_PFF_TOT_BS_df = pd.read_csv(path + 'stats_PFF_TOT_BS_df.csv',index_col=0) 
df_stats_RS_ADV_BS_df = pd.read_csv(path + 'stats_RS_ADV_BS_df.csv',index_col=0) 
df_stats_PFF_ADV_BS_df = pd.read_csv(path + 'Stats_PFF_ADV_BS_df.csv',index_col=0) 
df_player_extra_info_df = pd.read_csv(path + 'player_extra_info_df.csv',index_col=0)

#College DataFrames
# college_stats_RS_PG_BS_df = pd.read_csv(path + 'college_stats_RS_PG_BS_df.csv',index_col=0) 
# college_stats_CONF_PG_BS_df = pd.read_csv(path + 'college_stats_CONF_PG_BS_df.csv',index_col=0) 
# college_stats_RS_TOT_BS_df = pd.read_csv(path + 'college_stats_RS_TOT_BS_df.csv',index_col=0) 
# college_stats_CONF_TOT_BS_df = pd.read_csv(path + 'college_stats_CONF_TOT_BS_df.csv',index_col=0) 
# college_stats_RS_ADV_BS_df = pd.read_csv(path + 'college_stats_RS_ADV_BS_df.csv',index_col=0) 
# college_stats_CONF_ADV_BS_df = pd.read_csv(path + 'college_stats_CONF_ADV_BS_df.csv',index_col=0)     
# college_player_extra_info_df = pd.read_csv(path + 'college_player_extra_info_df.csv',index_col=0)

# 1. Estadísticas

Limpieza de las tablas qué contienen informacionies estadísticas por juego, totales y avanzados.

## 1.1 Estadísticas Por Juego - Temporada

In [4]:
#df_stats_RS_PG_BS_df.dtypes

In [5]:
#df_stats_RS_PG_BS_df.isna().sum()

In [6]:
#df_stats_RS_PG_BS_df[df_stats_RS_PG_BS_df['eFG%'].isna()]

In [7]:
#print(df_stats_RS_PG_BS_df.isna().sum()/len(df_stats_RS_PG_BS_df))

In [8]:
#df_stats_RS_PG_BS_df['SEASON'].unique()

Obtener las filas no nulas de la columna SEASON.

In [9]:
df_stats_RS_PG_BS_df = df_stats_RS_PG_BS_df[df_stats_RS_PG_BS_df['SEASON'].notna()]

Rellena los campos nulos con cero.

In [10]:
df_stats_RS_PG_BS_df[['FG%', '3P%', '2P%', 'eFG%', 'FT%']] = df_stats_RS_PG_BS_df[['FG%', '3P%', '2P%', 'eFG%', 'FT%']].fillna(0.000)

Almacena las líneas cuando el jugador no ha jugado en una nueva tabla e elimina estas líneas de la tabla original.

In [11]:
df_stats_RS_PG_BS_Did_Not_Play = df_stats_RS_PG_BS_df[df_stats_RS_PG_BS_df['TEAM'].map(lambda x: x.startswith('Did Not Play'))].copy()
df_stats_RS_PG_BS_df.drop(index=df_stats_RS_PG_BS_df[df_stats_RS_PG_BS_df['TEAM'].map(lambda x: x.startswith('Did Not Play'))].index, inplace=True)

Convierte datos de edad y número de juegos a valores enteros.

In [12]:
df_stats_RS_PG_BS_df[['AGE', 'G', 'GS']] = df_stats_RS_PG_BS_df[['AGE', 'G', 'GS']].applymap(lambda x: int(pd.to_numeric(x)))

Convierte columnas de MP a PTS de string a Float.

In [13]:
df_stats_RS_PG_BS_df.loc[:, 7:-1] = df_stats_RS_PG_BS_df[df_stats_RS_PG_BS_df.columns[7:-1]].applymap(lambda x: pd.to_numeric(x))

Visualización de líneas duplicadas.

In [14]:
#df_stats_RS_PG_BS_df[df_stats_RS_PG_BS_df.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [15]:
df_stats_RS_PG_BS_df = df_stats_RS_PG_BS_df.drop_duplicates()

Visualización de el df.

In [16]:
df_stats_RS_PG_BS_df

,SEASON,AGE,TEAM,LEAGUE,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,URL
0,1990-91,22,POR,NBA,PF,43,0,6.7,1.3,2.7,0.474,0.0,0.0,0.00,1.3,2.7,0.474,0.474,0.6,1.0,0.568,0.6,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,/players/a/abdelal01.html
1,1991-92,23,POR,NBA,PF,71,1,13.2,2.5,5.1,0.493,0.0,0.0,0.00,2.5,5.1,0.493,0.493,1.1,1.4,0.752,1.1,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,/players/a/abdelal01.html
2,1992-93,24,TOT,NBA,PF,75,52,17.5,3.3,6.3,0.518,0.0,0.0,0.00,3.3,6.3,0.519,0.518,1.2,1.5,0.759,1.7,2.8,4.5,0.4,0.3,0.3,1.3,2.5,7.7,/players/a/abdelal01.html
3,1992-93,24,MIL,NBA,PF,12,0,13.3,2.2,4.7,0.464,0.0,0.1,0.00,2.2,4.6,0.473,0.464,1.0,1.3,0.750,1.0,2.1,3.1,0.8,0.5,0.3,1.1,2.0,5.3,/players/a/abdelal01.html
4,1992-93,24,BOS,NBA,PF,63,52,18.3,3.5,6.6,0.525,0.0,0.0,0.00,3.5,6.6,0.525,0.525,1.2,1.6,0.760,1.8,3.0,4.8,0.3,0.3,0.3,1.3,2.6,8.2,/players/a/abdelal01.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20112,2018-19,21,LAL,NBA,C,33,12,15.6,3.4,5.8,0.580,0.0,0.0,0.00,3.4,5.8,0.580,0.580,1.7,2.0,0.864,1.6,3.3,4.9,0.8,0.1,0.8,1.0,2.2,8.5,/players/z/zubaciv01.html
20113,2018-19,21,LAC,NBA,C,26,25,20.2,3.8,7.2,0.538,0.0,0.0,0.00,3.8,7.2,0.538,0.538,1.7,2.3,0.733,2.3,5.3,7.7,1.5,0.4,0.9,1.4,2.5,9.4,/players/z/zubaciv01.html
20114,2019-20,22,LAC,NBA,C,72,70,18.4,3.3,5.3,0.613,0.0,0.0,0.00,3.3,5.3,0.616,0.613,1.7,2.3,0.747,2.7,4.8,7.5,1.1,0.2,0.9,0.8,2.3,8.3,/players/z/zubaciv01.html
20115,2020-21,23,LAC,NBA,C,72,33,22.3,3.6,5.5,0.652,0.0,0.1,0.25,3.6,5.4,0.656,0.654,1.9,2.4,0.789,2.6,4.6,7.2,1.3,0.3,0.9,1.1,2.6,9.0,/players/z/zubaciv01.html


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [17]:
df_stats_RS_PG_BS_df.to_excel('./Clean_data/Data_Clean_V1/Stats_RS_PG_BS_df_Clean.xlsx', index=False)
df_stats_RS_PG_BS_df.to_csv('./Clean_data/Data_Clean_V1/Stats_RS_PG_BS_df_Clean.csv', index=False)

## 1.2 Estadísticas Por Juego - Torneo de Conferencia

In [18]:
#df_stats_PFF_PG_BS_df.dtypes

In [19]:
#df_stats_PFF_PG_BS_df.isna().sum()

In [20]:
#df_stats_PFF_PG_BS_df[df_stats_PFF_PG_BS_df['3P%'].isna()]

In [21]:
#print(df_stats_PFF_PG_BS_df.isna().sum()/len(df_stats_PFF_PG_BS_df))

In [22]:
#df_stats_RS_PG_BS_df['SEASON'].unique()

Obtener las filas no nulas de la columna SEASON.

In [23]:
df_stats_PFF_PG_BS_df = df_stats_PFF_PG_BS_df[df_stats_PFF_PG_BS_df['SEASON'].notna()]

Rellena los campos nulos con cero.

In [24]:
df_stats_PFF_PG_BS_df[['FG%', '2P%', 'eFG%', 'FT%', '3P%']] = df_stats_PFF_PG_BS_df[['FG%', '2P%', 'eFG%', 'FT%', '3P%']].fillna(0.000)

Rellena los campos nulos con infomación valida.

In [25]:
df_stats_PFF_PG_BS_df.loc[846, 'POS'] = 'C'
df_stats_PFF_PG_BS_df.loc[3360, 'POS'] = 'PF'

Convierte datos de edad y número de juegos a valores enteros.

In [26]:
df_stats_PFF_PG_BS_df[['AGE', 'G', 'GS']] = df_stats_PFF_PG_BS_df[['AGE', 'G', 'GS']].applymap(lambda x: int((x)))

Visualización de líneas duplicadas.

In [27]:
#df_stats_PFF_PG_BS_df[df_stats_PFF_PG_BS_df.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [28]:
df_stats_PFF_PG_BS_df = df_stats_PFF_PG_BS_df.drop_duplicates()

Visualización de el ´df´.

In [29]:
df_stats_PFF_PG_BS_df

,SEASON,AGE,TEAM,LEAGUE,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,URL,3P%
0,1990-91,22,POR,NBA,PF,5,0,2.6,0.4,1.2,0.333,0.0,0.0,0.4,1.2,0.333,0.333,0.0,0.0,0.000,0.2,0.4,0.6,0.0,0.0,0.0,0.0,0.0,0.8,/players/a/abdelal01.html,0.000
1,1991-92,23,POR,NBA,PF,8,0,3.1,0.6,1.3,0.500,0.0,0.0,0.6,1.3,0.500,0.500,0.3,0.5,0.500,0.0,0.5,0.5,0.3,0.0,0.0,0.3,0.5,1.5,/players/a/abdelal01.html,0.000
2,1992-93,24,BOS,NBA,PF,4,4,17.0,2.8,6.0,0.458,0.0,0.0,2.8,6.0,0.458,0.458,0.0,0.0,0.000,0.5,2.8,3.3,0.3,0.0,0.3,2.3,1.8,5.5,/players/a/abdelal01.html,0.000
3,1993-94,24,DEN,NBA,PG,12,12,28.3,4.8,12.8,0.370,1.0,3.1,3.8,9.8,0.385,0.409,2.4,2.6,0.935,0.3,1.3,1.5,2.5,0.4,0.1,1.2,2.4,12.9,/players/a/abdulma02.html,0.324
4,1994-95,25,DEN,NBA,PG,3,2,25.3,4.0,11.0,0.364,0.7,4.0,3.3,7.0,0.476,0.394,4.7,4.7,1.000,0.7,1.0,1.7,1.7,0.7,0.0,2.7,2.7,13.3,/players/a/abdulma02.html,0.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6447,2016-17,22,CHI,NBA,SF,6,0,22.7,2.5,5.5,0.455,1.0,2.7,1.5,2.8,0.529,0.545,1.3,1.3,1.000,0.5,3.0,3.5,0.5,0.2,0.2,1.2,2.0,7.3,/players/z/zipsepa01.html,0.375
6448,2017-18,21,CLE,NBA,C,8,0,2.9,0.6,1.3,0.500,0.0,0.0,0.6,1.3,0.500,0.500,0.4,0.8,0.500,0.1,0.6,0.8,0.1,0.0,0.1,0.3,0.3,1.6,/players/z/zizican01.html,0.000
6449,2018-19,21,LAC,NBA,C,4,3,9.8,2.3,4.5,0.500,0.0,0.0,2.3,4.5,0.500,0.500,0.5,0.8,0.667,2.8,2.8,5.5,0.3,0.5,0.5,1.0,1.3,5.0,/players/z/zubaciv01.html,0.000
6450,2019-20,22,LAC,NBA,C,13,13,24.6,3.4,6.0,0.564,0.0,0.0,3.4,6.0,0.564,0.564,2.3,2.8,0.811,2.4,4.8,7.2,0.6,0.2,0.8,0.9,2.6,9.1,/players/z/zubaciv01.html,0.000


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [30]:
df_stats_PFF_PG_BS_df.to_excel('./Clean_data/Data_Clean_V1/Stats_PFF_PG_BS_df_Clean.xlsx', index=False)
df_stats_PFF_PG_BS_df.to_csv('./Clean_data/Data_Clean_V1/Stats_PFF_PG_BS_df_Clean.csv', index=False)

## 1.3 Estadísticas Totales - Temporada

In [31]:
#df_stats_RS_TOT_BS_df.dtypes

In [32]:
#df_stats_RS_TOT_BS_df.isna().sum()

In [33]:
#df_stats_RS_TOT_BS_df[df_stats_RS_TOT_BS_df['FG'].isna()]

In [34]:
#df_stats_RS_TOT_BS_df[df_stats_RS_TOT_BS_df['Trp Dbl'].isna()]

In [35]:
#print(df_stats_RS_TOT_BS_df.isna().sum()/len(df_stats_RS_TOT_BS_df))

In [36]:
#df_stats_RS_TOT_BS_df['SEASON'].unique()

Obtener las filas no nulas de la columna SEASON.

In [37]:
df_stats_RS_TOT_BS_df = df_stats_RS_TOT_BS_df[df_stats_RS_TOT_BS_df['SEASON'].notna()]

Rellena los campos nulos con cero.

In [38]:
df_stats_RS_TOT_BS_df[['FG%', '3P%', '2P%', 'eFG%', 'FT%'] ] = df_stats_RS_TOT_BS_df[['FG%', '3P%', '2P%', 'eFG%', 'FT%']].fillna(0.000)

Elimina la columna Trp Dbl ya que es 79% nula.

In [39]:
df_stats_RS_TOT_BS_df.drop(columns=['Trp Dbl'], inplace=True)

Convierte datos de edad y número de juegos a valores enteros.

In [40]:
df_stats_RS_TOT_BS_df[['AGE', 'G', 'GS']] = df_stats_RS_TOT_BS_df[['AGE', 'G', 'GS']].applymap(lambda x: int((x)))

Visualización de líneas duplicadas.

In [41]:
#df_stats_RS_TOT_BS_df[df_stats_RS_TOT_BS_df.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [42]:
df_stats_RS_TOT_BS_df = df_stats_RS_TOT_BS_df.drop_duplicates()

Visualización de el ´df´.

In [43]:
df_stats_RS_TOT_BS_df

,SEASON,AGE,TEAM,LEAGUE,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,URL
0,1990-91,22,POR,NBA,PF,43,0,290.0,55.0,116.0,0.474,0.0,0.0,0.00,55.0,116.0,0.474,0.474,25.0,44.0,0.568,27.0,62.0,89.0,12.0,4.0,12.0,22.0,39.0,135.0,/players/a/abdelal01.html
1,1991-92,23,POR,NBA,PF,71,1,934.0,178.0,361.0,0.493,0.0,0.0,0.00,178.0,361.0,0.493,0.493,76.0,101.0,0.752,81.0,179.0,260.0,30.0,25.0,16.0,66.0,132.0,432.0,/players/a/abdelal01.html
2,1992-93,24,TOT,NBA,PF,75,52,1311.0,245.0,473.0,0.518,0.0,1.0,0.00,245.0,472.0,0.519,0.518,88.0,116.0,0.759,126.0,211.0,337.0,27.0,25.0,26.0,97.0,189.0,578.0,/players/a/abdelal01.html
3,1992-93,24,MIL,NBA,PF,12,0,159.0,26.0,56.0,0.464,0.0,1.0,0.00,26.0,55.0,0.473,0.464,12.0,16.0,0.750,12.0,25.0,37.0,10.0,6.0,4.0,13.0,24.0,64.0,/players/a/abdelal01.html
4,1992-93,24,BOS,NBA,PF,63,52,1152.0,219.0,417.0,0.525,0.0,0.0,0.00,219.0,417.0,0.525,0.525,76.0,100.0,0.760,114.0,186.0,300.0,17.0,19.0,22.0,84.0,165.0,514.0,/players/a/abdelal01.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18920,2018-19,21,LAL,NBA,C,33,12,516.0,112.0,193.0,0.580,0.0,0.0,0.00,112.0,193.0,0.580,0.580,57.0,66.0,0.864,54.0,108.0,162.0,25.0,4.0,27.0,33.0,73.0,281.0,/players/z/zubaciv01.html
18921,2018-19,21,LAC,NBA,C,26,25,524.0,100.0,186.0,0.538,0.0,0.0,0.00,100.0,186.0,0.538,0.538,44.0,60.0,0.733,61.0,139.0,200.0,38.0,10.0,24.0,37.0,64.0,244.0,/players/z/zubaciv01.html
18922,2019-20,22,LAC,NBA,C,72,70,1326.0,236.0,385.0,0.613,0.0,2.0,0.00,236.0,383.0,0.616,0.613,124.0,166.0,0.747,197.0,346.0,543.0,82.0,16.0,66.0,61.0,168.0,596.0,/players/z/zubaciv01.html
18923,2020-21,23,LAC,NBA,C,72,33,1609.0,257.0,394.0,0.652,1.0,4.0,0.25,256.0,390.0,0.656,0.654,135.0,171.0,0.789,189.0,330.0,519.0,90.0,24.0,62.0,81.0,187.0,650.0,/players/z/zubaciv01.html


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [44]:
df_stats_RS_TOT_BS_df.to_excel('./Clean_data/Data_Clean_V1/Stats_RS_TOT_BS_df_Clean.xlsx', index=False)
df_stats_RS_TOT_BS_df.to_csv('./Clean_data/Data_Clean_V1/Stats_RS_TOT_BS_df_Clean.csv', index=False)

## 1.4 Estadísticas Totales - Torneo de Conferencia

In [45]:
#df_stats_PFF_TOT_BS_df.dtypes

In [46]:
#df_stats_PFF_TOT_BS_df.isna().sum()

In [47]:
#df_stats_PFF_TOT_BS_df[df_stats_PFF_TOT_BS_df['eFG%'].isna()]

In [48]:
#df_stats_PFF_TOT_BS_df[df_stats_PFF_TOT_BS_df['2P%'].isna()]

In [49]:
#print(df_stats_PFF_TOT_BS_df.isna().sum()/len(df_stats_PFF_TOT_BS_df))

In [50]:
#df_stats_PFF_TOT_BS_df['SEASON'].unique()

Obtener las filas no nulas de la columna SEASON.

In [51]:
df_stats_PFF_TOT_BS_df = df_stats_PFF_TOT_BS_df[df_stats_PFF_TOT_BS_df['SEASON'].notna()]

Rellena los campos nulos con cero.

In [52]:
df_stats_PFF_TOT_BS_df[['FG%', '2P%', 'eFG%', 'FT%', '3P%']] = df_stats_PFF_TOT_BS_df[['FG%', '2P%', 'eFG%', 'FT%', '3P%']].fillna(0.000)

Rellena los campos nulos con infomación valida.

In [53]:
df_stats_PFF_TOT_BS_df.loc[846, 'POS'] = 'C'
df_stats_PFF_TOT_BS_df.loc[3364, 'POS'] = 'PF'

Elimina la columna Trp Dbl ya que es 94% nula.

In [54]:
df_stats_PFF_TOT_BS_df.drop(columns=['Trp Dbl'], inplace=True)

Convierte datos de edad y número de juegos a valores enteros.

In [55]:
df_stats_PFF_TOT_BS_df[['AGE', 'G', 'GS']] = df_stats_PFF_TOT_BS_df[['AGE', 'G', 'GS']].applymap(lambda x: int((x)))

Visualización de líneas duplicadas.

In [56]:
#df_stats_PFF_TOT_BS_df[df_stats_PFF_TOT_BS_df.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [57]:
df_stats_PFF_TOT_BS_df = df_stats_PFF_TOT_BS_df.drop_duplicates()

Visualización de el ´df´.

In [58]:
df_stats_PFF_TOT_BS_df

,SEASON,AGE,TEAM,LEAGUE,POS,G,GS,MP,FG,FGA,FG%,3P,3PA,2P,2PA,2P%,eFG%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,URL,3P%
0,1990-91,22,POR,NBA,PF,5,0,13.0,2.0,6.0,0.333,0.0,0.0,2.0,6.0,0.333,0.333,0.0,0.0,0.000,1.0,2.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,/players/a/abdelal01.html,0.000
1,1991-92,23,POR,NBA,PF,8,0,25.0,5.0,10.0,0.500,0.0,0.0,5.0,10.0,0.500,0.500,2.0,4.0,0.500,0.0,4.0,4.0,2.0,0.0,0.0,2.0,4.0,12.0,/players/a/abdelal01.html,0.000
2,1992-93,24,BOS,NBA,PF,4,4,68.0,11.0,24.0,0.458,0.0,0.0,11.0,24.0,0.458,0.458,0.0,0.0,0.000,2.0,11.0,13.0,1.0,0.0,1.0,9.0,7.0,22.0,/players/a/abdelal01.html,0.000
3,1993-94,24,DEN,NBA,PG,12,12,339.0,57.0,154.0,0.370,12.0,37.0,45.0,117.0,0.385,0.409,29.0,31.0,0.935,3.0,15.0,18.0,30.0,5.0,1.0,14.0,29.0,155.0,/players/a/abdulma02.html,0.324
4,1994-95,25,DEN,NBA,PG,3,2,76.0,12.0,33.0,0.364,2.0,12.0,10.0,21.0,0.476,0.394,14.0,14.0,1.000,2.0,3.0,5.0,5.0,2.0,0.0,8.0,8.0,40.0,/players/a/abdulma02.html,0.167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6451,2016-17,22,CHI,NBA,SF,6,0,136.0,15.0,33.0,0.455,6.0,16.0,9.0,17.0,0.529,0.545,8.0,8.0,1.000,3.0,18.0,21.0,3.0,1.0,1.0,7.0,12.0,44.0,/players/z/zipsepa01.html,0.375
6452,2017-18,21,CLE,NBA,C,8,0,23.0,5.0,10.0,0.500,0.0,0.0,5.0,10.0,0.500,0.500,3.0,6.0,0.500,1.0,5.0,6.0,1.0,0.0,1.0,2.0,2.0,13.0,/players/z/zizican01.html,0.000
6453,2018-19,21,LAC,NBA,C,4,3,39.0,9.0,18.0,0.500,0.0,0.0,9.0,18.0,0.500,0.500,2.0,3.0,0.667,11.0,11.0,22.0,1.0,2.0,2.0,4.0,5.0,20.0,/players/z/zubaciv01.html,0.000
6454,2019-20,22,LAC,NBA,C,13,13,320.0,44.0,78.0,0.564,0.0,0.0,44.0,78.0,0.564,0.564,30.0,37.0,0.811,31.0,62.0,93.0,8.0,2.0,10.0,12.0,34.0,118.0,/players/z/zubaciv01.html,0.000


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [59]:
df_stats_PFF_TOT_BS_df.to_excel('./Clean_data/Data_Clean_V1/Stats_PFF_TOT_BS_df_Clean.xlsx', index=False)
df_stats_PFF_TOT_BS_df.to_csv('./Clean_data/Data_Clean_V1/Stats_PFF_TOT_BS_df_Clean.csv', index=False)

## 1.5 Estadísticas Avanzadas - Temporada



In [60]:
#df_stats_RS_ADV_BS_df.dtypes

In [61]:
#df_stats_RS_ADV_BS_df.isna().sum()

In [62]:
#df_stats_RS_ADV_BS_df[df_stats_RS_ADV_BS_df['PER'].isna()]

In [63]:
#df_stats_RS_ADV_BS_df[df_stats_RS_ADV_BS_df['WS/48'].isna()]

In [64]:
#print(df_stats_RS_ADV_BS_df.isna().sum()/len(df_stats_RS_ADV_BS_df))

In [65]:
#df_stats_RS_ADV_BS_df['SEASON'].unique()

Rellena los campos nulos con cero.

In [66]:
columns_to_fill = ['PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'WS/48']
df_stats_RS_ADV_BS_df[columns_to_fill] = df_stats_RS_ADV_BS_df[columns_to_fill].fillna(0.0)

Convierte datos de edad y número de juegos a valores enteros.

In [67]:
df_stats_RS_ADV_BS_df[['AGE', 'G']] = df_stats_RS_ADV_BS_df[['AGE', 'G']].applymap(lambda x: int((x)))

Visualización de líneas duplicadas.

In [68]:
#df_stats_RS_ADV_BS_df[df_stats_RS_ADV_BS_df.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [69]:
df_stats_RS_ADV_BS_df = df_stats_RS_ADV_BS_df.drop_duplicates()

Visualización de el ´df´.

In [70]:
df_stats_RS_ADV_BS_df

,SEASON,AGE,TEAM,LEAGUE,POS,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,URL
0,1990-91,22,POR,NBA,PF,43,290.0,13.1,0.499,0.000,0.379,10.4,23.4,17.0,5.8,0.7,2.5,14.0,22.1,0.0,0.5,0.5,0.079,-3.4,-1.2,-4.6,-0.2,/players/a/abdelal01.html
1,1991-92,23,POR,NBA,PF,71,934.0,13.5,0.533,0.000,0.280,9.5,20.9,15.2,4.7,1.3,1.1,14.0,20.6,0.6,1.5,2.1,0.110,-2.3,-0.4,-2.6,-0.1,/players/a/abdelal01.html
2,1992-93,24,TOT,NBA,PF,75,1311.0,13.2,0.551,0.002,0.245,11.0,18.1,14.6,3.0,1.0,1.2,15.6,20.5,0.7,1.3,2.0,0.074,-2.4,-1.5,-3.9,-0.6,/players/a/abdelal01.html
3,1992-93,24,MIL,NBA,PF,12,159.0,11.9,0.508,0.018,0.286,8.6,18.3,13.4,9.5,1.9,1.6,17.1,20.5,0.0,0.2,0.1,0.035,-3.3,-0.2,-3.5,-0.1,/players/a/abdelal01.html
4,1992-93,24,BOS,NBA,PF,63,1152.0,13.4,0.557,0.000,0.240,11.3,18.1,14.8,2.2,0.8,1.2,15.4,20.5,0.7,1.2,1.9,0.079,-2.3,-1.7,-4.0,-0.6,/players/a/abdelal01.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18895,2018-19,21,LAL,NBA,C,33,516.0,19.5,0.633,0.000,0.342,11.0,21.2,16.2,7.3,0.4,4.3,12.9,20.5,1.1,0.6,1.8,0.165,-0.3,0.1,-0.2,0.2,/players/z/zubaciv01.html
18896,2018-19,21,LAC,NBA,C,26,524.0,18.3,0.574,0.000,0.323,12.7,27.2,20.2,10.9,0.9,3.6,14.8,20.1,0.7,0.7,1.4,0.132,-0.2,-0.5,-0.7,0.2,/players/z/zubaciv01.html
18897,2019-20,22,LAC,NBA,C,72,1326.0,21.7,0.651,0.005,0.431,15.9,26.4,21.3,9.1,0.6,4.4,11.8,16.4,4.4,2.3,6.6,0.241,1.9,0.9,2.8,1.6,/players/z/zubaciv01.html
18898,2020-21,23,LAC,NBA,C,72,1609.0,19.1,0.693,0.010,0.434,13.6,22.4,18.1,7.9,0.7,3.4,14.7,15.1,4.8,2.1,6.9,0.206,0.6,0.4,1.0,1.2,/players/z/zubaciv01.html


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [71]:
df_stats_RS_ADV_BS_df.to_excel('./Clean_data/Data_Clean_V1/Stats_RS_ADV_BS_df_Clean.xlsx', index=False)
df_stats_RS_ADV_BS_df.to_csv('./Clean_data/Data_Clean_V1/Stats_RS_ADV_BS_df_Clean.csv', index=False)

## 1.6 Estadísticas Avanzadas - Torneo de Conferencia

In [72]:
#df_stats_PFF_ADV_BS_df.dtypes

In [73]:
#df_stats_PFF_ADV_BS_df.isna().sum()

In [74]:
#df_stats_PFF_ADV_BS_df[df_stats_PFF_ADV_BS_df['FTr'].isna()]

In [75]:
#df_stats_PFF_ADV_BS_df[df_stats_PFF_ADV_BS_df['SEASON'].isna()]

In [76]:
#print(df_stats_PFF_ADV_BS_df.isna().sum()/len(df_stats_PFF_ADV_BS_df))

In [77]:
#df_stats_PFF_ADV_BS_df.loc[3362:3367]

In [78]:
#df_stats_PFF_ADV_BS_df['SEASON'].unique()

Obtener las filas no nulas de la columna SEASON.

In [79]:
df_stats_PFF_ADV_BS_df = df_stats_PFF_ADV_BS_df[df_stats_PFF_ADV_BS_df['SEASON'].notna()]

Rellena los campos nulos con cero.

In [80]:
columns_to_fill2 = ['PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'WS/48']
df_stats_PFF_ADV_BS_df[columns_to_fill2] = df_stats_PFF_ADV_BS_df[columns_to_fill2].fillna(0.0)

Rellena los campos nulos con infomación valida.

In [81]:
df_stats_PFF_ADV_BS_df.loc[846, 'POS'] = 'C'
df_stats_PFF_ADV_BS_df.loc[3362, 'POS'] = 'PF'

Convierte datos de edad y número de juegos a valores enteros.

In [82]:
df_stats_PFF_ADV_BS_df[['AGE', 'G']] = df_stats_PFF_ADV_BS_df[['AGE', 'G']].applymap(lambda x: int((x)))

Visualización de líneas duplicadas.

In [83]:
#df_stats_PFF_ADV_BS_df[df_stats_PFF_ADV_BS_df.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [84]:
df_stats_PFF_ADV_BS_df = df_stats_PFF_ADV_BS_df.drop_duplicates()

Visualización de el ´df´.

In [85]:
df_stats_PFF_ADV_BS_df

,SEASON,AGE,TEAM,LEAGUE,POS,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,URL
0,1990-91,22,POR,NBA,PF,5,13.0,6.3,0.333,0.000,0.000,8.8,18.9,13.7,0.0,0.0,0.0,0.0,19.8,0.0,0.0,0.0,-0.035,-2.6,-4.6,-7.3,0.0,/players/a/abdelal01.html
1,1991-92,23,POR,NBA,PF,8,25.0,8.6,0.510,0.000,0.400,0.0,19.4,9.8,12.9,0.0,0.0,14.5,23.7,0.0,0.0,0.0,-0.005,-4.2,-0.4,-4.6,0.0,/players/a/abdelal01.html
2,1992-93,24,BOS,NBA,PF,4,68.0,3.2,0.458,0.000,0.000,3.7,20.6,12.1,2.2,0.0,0.9,27.3,21.9,-0.2,0.0,-0.2,-0.145,-8.4,-2.6,-11.0,-0.1,/players/a/abdelal01.html
3,1993-94,24,DEN,NBA,PG,12,339.0,11.8,0.462,0.240,0.201,1.1,5.0,3.1,18.0,0.8,0.2,7.7,24.8,0.2,0.1,0.3,0.042,-0.8,-1.9,-2.7,-0.1,/players/a/abdulma02.html
4,1994-95,25,DEN,NBA,PG,3,76.0,10.2,0.511,0.364,0.424,3.0,5.1,4.0,12.3,1.4,0.0,17.0,27.8,0.0,0.0,-0.1,-0.048,-1.1,-3.8,-4.9,-0.1,/players/a/abdulma02.html
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6446,2016-17,22,CHI,NBA,SF,6,136.0,8.5,0.602,0.485,0.242,2.5,14.6,8.7,3.4,0.4,0.8,16.1,14.4,0.1,0.1,0.1,0.048,-2.9,-0.7,-3.6,-0.1,/players/z/zipsepa01.html
6447,2017-18,21,CLE,NBA,C,8,23.0,14.9,0.514,0.000,0.600,5.1,26.5,15.5,8.0,0.0,4.0,13.7,29.5,0.0,0.0,0.0,-0.001,-2.2,-2.2,-4.4,0.0,/players/z/zizican01.html
6448,2018-19,21,LAC,NBA,C,4,39.0,22.9,0.518,0.000,0.167,29.6,33.4,31.4,4.3,2.4,4.3,17.2,24.6,0.0,0.0,0.1,0.067,2.2,-2.4,-0.2,0.0,/players/z/zubaciv01.html
6449,2019-20,22,LAC,NBA,C,13,320.0,16.0,0.626,0.000,0.474,10.7,21.9,16.2,3.5,0.3,3.0,11.3,14.5,0.7,0.3,1.0,0.151,0.0,-0.5,-0.5,0.1,/players/z/zubaciv01.html


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [86]:
df_stats_PFF_ADV_BS_df.to_excel('./Clean_data/Data_Clean_V1/Stats_PFF_ADV_BS_df_Clean.xlsx', index=False)
df_stats_PFF_ADV_BS_df.to_csv('./Clean_data/Data_Clean_V1/Stats_PFF_ADV_BS_df_Clean.csv', index=False)

## 2. Información Adicional - Jugador

Limpieza de las tablas qué contienen informacionies adicionales por jugador.

In [87]:
#df_player_extra_info_df.dtypes

In [88]:
#df_player_extra_info_df.isna().sum()

In [89]:
#df_player_extra_info_df2[df_player_extra_info_df2['NBA_CHAMP'] == 0]

In [90]:
#print(df_player_extra_info_df.isna().sum()/len(df_player_extra_info_df))

In [91]:
#df_player_extra_info_df2['SUM_YEAR_NBA_CHAMP'].unique()

Copie df1 a uno nuevo llamado df2.

In [92]:
df_player_extra_info_df2 = df_player_extra_info_df.copy()

Visualización de líneas duplicadas.

In [93]:
#df_player_extra_info_df2[df_player_extra_info_df2.duplicated()]

Eliminar líneas duplicadas de la tabla.

In [94]:
df_player_extra_info_df2 = df_player_extra_info_df2.drop_duplicates()

Esta función asigna objetos de columna a enteros. Ejemplo: 
- `nan = 0`
- `'2020-21' = 1`
- `'2' = 2`
- `'3' = 3`

In [95]:
def translate_values(x):  
    if pd.isnull(x):
        return 0
    
    if isinstance(x, float) or '-' not in x:
        return int(x)

    return 1

Lista de columnas para traducir valores.

In [96]:
translate_cols = ['ALL-DEFENSIVE', 'ALL-NBA', 'ALL-ROOKIE', 'ALL_STAR', 'AST_CHAMP', 'AS_MVP','BLK_CHAMP', 'DEF._POY', 'FINALS_MVP', 
                  'HALL_OF_FAME', 'MOST_IMPROVED', 'MVP', 'ROY', 'SCORING_CHAMP', 'SIXTH_MAN', 'STL_CHAMP', 'TRB_CHAMP', 'NBA_CHAMP', 
                  'NBA_75TH_ANNIV._TEAM', '1986_NBA_CHAMP', '1987_NBA_CHAMP', '1988_NBA_CHAMP', '1989_NBA_CHAMP', '1990_NBA_CHAMP', 
                  '1991_NBA_CHAMP', '1993_NBA_CHAMP', '1994_NBA_CHAMP', '1995_NBA_CHAMP', '1996_NBA_CHAMP', '1997_NBA_CHAMP', 
                  '1998_NBA_CHAMP', '1999_NBA_CHAMP', '2000_NBA_CHAMP', '2001_NBA_CHAMP', '2002_NBA_CHAMP', '2003_NBA_CHAMP', 
                  '2004_NBA_CHAMP', '2005_NBA_CHAMP', '2006_NBA_CHAMP', '2007_NBA_CHAMP', '2008_NBA_CHAMP', '2009_NBA_CHAMP', 
                  '2010_NBA_CHAMP', '2011_NBA_CHAMP', '2012_NBA_CHAMP', '2013_NBA_CHAMP', '2014_NBA_CHAMP', '2015_NBA_CHAMP', 
                  '2016_NBA_CHAMP', '2017_NBA_CHAMP', '2018_NBA_CHAMP', '2019_NBA_CHAMP', '2020_NBA_CHAMP', '2021_NBA_CHAMP']

Aplique la función de traducción en las columnas seleccionadas arriba.

In [97]:
df_player_extra_info_df2[translate_cols] = df_player_extra_info_df2[translate_cols].applymap(lambda x: translate_values(x))

Rellena los campos nulos `COLLEGE_URL` llenar con una istring vacía.

In [98]:
df_player_extra_info_df2['COLLEGE_URL'] = df_player_extra_info_df2['COLLEGE_URL'].fillna('')

Ordenar y mostrar columnas.

In [99]:
#columns = df_player_extra_info_df2.columns.to_list()
#columns.sort()
#columns

Reordenando las columnas

In [100]:
df_player_extra_info_df2 = df_player_extra_info_df2[['URL', 'COLLEGE_URL', 'ALL-DEFENSIVE', 'ALL-NBA', 'ALL-ROOKIE', 'ALL_STAR', 'AST_CHAMP', 'AS_MVP', 
                                                    'BLK_CHAMP', 'DEF._POY', 'FINALS_MVP', 'HALL_OF_FAME', 'MOST_IMPROVED', 'MVP', 'ROY', 'SCORING_CHAMP',
                                                    'SIXTH_MAN', 'STL_CHAMP', 'TRB_CHAMP', 'NBA_75TH_ANNIV._TEAM', 'NBA_CHAMP', '1986_NBA_CHAMP', '1987_NBA_CHAMP',
                                                    '1988_NBA_CHAMP', '1989_NBA_CHAMP', '1990_NBA_CHAMP', '1991_NBA_CHAMP', '1993_NBA_CHAMP', '1994_NBA_CHAMP',
                                                    '1995_NBA_CHAMP', '1996_NBA_CHAMP', '1997_NBA_CHAMP', '1998_NBA_CHAMP', '1999_NBA_CHAMP', '2000_NBA_CHAMP',
                                                    '2001_NBA_CHAMP', '2002_NBA_CHAMP', '2003_NBA_CHAMP', '2004_NBA_CHAMP', '2005_NBA_CHAMP', '2006_NBA_CHAMP',
                                                    '2007_NBA_CHAMP', '2008_NBA_CHAMP', '2009_NBA_CHAMP', '2010_NBA_CHAMP', '2011_NBA_CHAMP', '2012_NBA_CHAMP',
                                                    '2013_NBA_CHAMP', '2014_NBA_CHAMP', '2015_NBA_CHAMP', '2016_NBA_CHAMP', '2017_NBA_CHAMP', '2018_NBA_CHAMP',
                                                    '2019_NBA_CHAMP', '2020_NBA_CHAMP', '2021_NBA_CHAMP']]


Creando una lista para las columnas del año NBA_CHAMP.

In [101]:
year_columns =[ '1986_NBA_CHAMP', '1987_NBA_CHAMP', '1988_NBA_CHAMP', '1989_NBA_CHAMP', '1990_NBA_CHAMP', '1991_NBA_CHAMP', '1993_NBA_CHAMP',
                          '1994_NBA_CHAMP', '1995_NBA_CHAMP', '1996_NBA_CHAMP', '1997_NBA_CHAMP', '1998_NBA_CHAMP', '1999_NBA_CHAMP', '2000_NBA_CHAMP', 
                          '2001_NBA_CHAMP', '2002_NBA_CHAMP', '2003_NBA_CHAMP', '2004_NBA_CHAMP', '2005_NBA_CHAMP', '2006_NBA_CHAMP', '2007_NBA_CHAMP',
                          '2008_NBA_CHAMP', '2009_NBA_CHAMP', '2010_NBA_CHAMP', '2011_NBA_CHAMP', '2012_NBA_CHAMP', '2013_NBA_CHAMP', '2014_NBA_CHAMP', 
                          '2015_NBA_CHAMP', '2016_NBA_CHAMP', '2017_NBA_CHAMP', '2018_NBA_CHAMP', '2019_NBA_CHAMP', '2020_NBA_CHAMP', '2021_NBA_CHAMP']

Suma de columnas `NBA_CHAMP` de 1986 a 2021 por jugador y crea una nueva columna llamada `SUM_YEAR_NBA_CHAMP`.

In [102]:
df_player_extra_info_df2['SUM_YEAR_NBA_CHAMP'] = df_player_extra_info_df2[year_columns].sum(axis=1)

Función que toma el año en el nombre de la columna donde el valor de la fila es 1.

In [103]:
def get_column_name(x):
    row_dict = dict(x)
    
    for key, value in row_dict.items():
        if value:
            return int(key[:4])
    return 0

Aplicando la función anterior y creando una nueva columna `YEAR_NBA_CHAMP`.

In [104]:
df_player_extra_info_df2['YEAR_NBA_CHAMP'] = df_player_extra_info_df2[year_columns].apply(lambda x: get_column_name(x), axis=1)

Eliminando columnas con `ano_NBA_CHAMP`.

In [105]:
df_player_extra_info_df2 = df_player_extra_info_df2.drop(columns=year_columns)

Agregando las columnas `NBA_CHAMP` y `SUM_YEAR_NBA_CHAMP`.

In [108]:
df_player_extra_info_df2['SUM_NBA_CHAMP'] = df_player_extra_info_df2[['NBA_CHAMP', 'SUM_YEAR_NBA_CHAMP']].sum(axis=1)

Visualización de el ´df´.

In [109]:
df_player_extra_info_df2

,URL,COLLEGE_URL,ALL-DEFENSIVE,ALL-NBA,ALL-ROOKIE,ALL_STAR,AST_CHAMP,AS_MVP,BLK_CHAMP,DEF._POY,FINALS_MVP,HALL_OF_FAME,MOST_IMPROVED,MVP,ROY,SCORING_CHAMP,SIXTH_MAN,STL_CHAMP,TRB_CHAMP,NBA_75TH_ANNIV._TEAM,NBA_CHAMP,SUM_YEAR_NBA_CHAMP,YEAR_NBA_CHAMP,SUM_NBA_CHAMP
0,/players/a/abdelal01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,/players/a/abdulma02.html,/players/mahmoud-abdul-rauf-1.html,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,/players/a/abdulta01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,/players/a/abdursh01.html,/players/shareef-abdur-rahim-1.html,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,/players/a/abrinal01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3024,/players/z/zimmede01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3025,/players/z/zimmest01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3026,/players/z/zipsepa01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3027,/players/z/zizican01.html,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Ahorrar y convierta el archivo a formatos csv y xlsx.

In [110]:
df_player_extra_info_df2.to_excel('./Clean_data/Data_Clean_V1/Player_extra_info_df_Clean.xlsx', index=False)
df_player_extra_info_df2.to_csv('./Clean_data/Data_Clean_V1/Player_extra_info_df_Clean.csv', index=False)